In [13]:
import taichi as ti
import taichi.math as tm

In [14]:
# Initializes the Taichi runtime.
ti.init(arch=ti.gpu, gdb_trigger=True)

"""
1. `arch`: Choose backend (cpu, cuda, metal, opengl)
2. `gdb_trigger`: Enable/disable GDB when Taichi crashes.
"""

[Taichi] Starting on arch=cuda


'\n1. `arch`: Choose backend (cpu, cuda, metal, opengl)\n2. `gdb_trigger`: Enable/disable GDB when Taichi crashes.\n'

In [15]:
n = 320
pixels = ti.field(dtype=float, shape=(n * 2, n))

# Create a dense two-dimensional array
# ti.filed is similar to numpy ndarray or pytorch tensor
# Utilize indices to iterate over a field

print(f"shape of pixels: {pixels.shape}")
print(f"first element of pixels: {pixels[0, 0]}")

shape of pixels: (640, 320)
first element of pixels: 0.0


In [16]:
@ti.func
def complex_sqr(z):  # complex square of a 2D vector
    return tm.vec2(z[0] * z[0] - z[1] * z[1], 2 * z[0] * z[1])

"""
Taichi function:
1. Can only be invoked within kernels or other Taichi functions.
2. Do not require type hinting
3. Corresponds to the __device__ in CUDA
"""

'\nTaichi function:\n1. Can only be invoked within kernels or other Taichi functions.\n2. Do not require type hinting\n3. Corresponds to the __device__ in CUDA\n'

In [17]:
@ti.kernel
def paint(t: float):
    for i, j in pixels:  # Parallelized over all pixels
        c = tm.vec2(-0.8, tm.cos(t) * 0.2)
        z = tm.vec2(i / n - 1, j / n - 0.5) * 2
        iterations = 0
        while z.norm() < 20 and iterations < 50:
            z = complex_sqr(z) + c
            iterations += 1
        pixels[i, j] = 1 - iterations * 0.02  #? Can we modify the pixels in for loop directly?

"""
Taichi kernel:
1. Serve as the entry points for Taichi to take over the execution
2. The arguments and return values of Taichi kernels must be type hinted
3. Corresponds to the __global__ in CUDA
4. Any for loop at the outermost scope within a kernel is automatically parallelized
"""

'\nTaichi kernel:\n1. Serve as the entry points for Taichi to take over the execution\n2. The arguments and return values of Taichi kernels must be type hinted\n3. Corresponds to the __global__ in CUDA\n4. Any for loop at the outermost scope within a kernel is automatically parallelized\n'

In [18]:
# Don't test due to the X11 issue in server
# gui = ti.GUI("Julia Set", res=(n * 2, n))

# i = 0
# while gui.running:
#     paint(i * 0.03)
#     gui.set_image(pixels)
#     gui.show()
#     i += 1

In [19]:
# !ti gallery

In [20]:
#!ti example -p fractal